# Example of computing vibrational energy levels of a hexatomic molecule using the Taylor series expansion approximation for the kinetic and potential energy operators

Compute vibrational states of $\text{H}_2\text{CO}$ molecule using the potential energy surface from
[AAl-Refaie, A. F., Yachmenev, A., Tennyson, J.Yurchenko, S. N., "ExoMol line lists VIII: A variationally computed line list for hot formaldehyde",
Monthly Notices of the Royal Astronomical Society 448, 1704-1714 (2015).]


In [2]:
import sys

sys.path.insert(1, "../")

from typing import Callable, List

import jax
import jax.numpy as jnp
import numpy as np
from numpy.polynomial.hermite import hermder, hermgauss, hermval
from scipy import optimize
from scipy.special import factorial
from vibrojet.keo import Gmat, com
from vibrojet.potentials import h2co_AYTY
from vibrojet.taylor import deriv_list

jax.config.update("jax_enable_x64", True)

Compute equilibrium coordinates, around which the Taylor series expansions will be built

In [3]:
potential = h2co_AYTY.poten
vmin = optimize.minimize(potential, [1.2, 1.1, 1.1, 120.0*np.pi/180,  120.0*np.pi/180, np.pi])
r0 = vmin.x
v0 = vmin.fun
print("Equilibrium coordinates:", r0)
print("Min of the potential:", v0)

Equilibrium coordinates: [1.19999999 1.09999999 1.09999999 2.13       2.13       3.14159265]
Min of the potential: 3.0383371388387545e-11


Define mapping function from internal valence coordinates, $r_{CO}$, $r_{CH1}$, $r_{CH2}$, $\alpha_{OCH1}$, $\alpha_{OCH2}$, and $\tau$, to Cartesian coordinates

In [4]:
masses = [12.0, 15.99491463, 1.00782505, 1.00782505] # masses of C, O, H, H
ncoo = len(r0)

@com(masses)
def internal_to_cartesian(internal_coords):
    rCO, rCH1, rCH2, aOCH1, aOCH2, tau = internal_coords
    xyz =[[0.0, 0.0, 0.0],
          [0.0, 0.0, rCO],
          [rCH1 * jnp.sin(aOCH1) * jnp.cos(tau * 0.5), -rCH1 * jnp.sin(aOCH1) * jnp.sin(tau * 0.5), rCH1 * jnp.cos(aOCH1)],
          [rCH2 * jnp.sin(aOCH2) * jnp.cos(tau * 0.5), rCH2 * jnp.sin(aOCH2) * jnp.sin(tau * 0.5), rCH2 * jnp.cos(aOCH2)]]
    return jnp.array(xyz, dtype=jnp.float64)
    

Utility function for generating index combinations corresponding to products of primitive basis functions as well as derivative indices used in the expansion of operators

In [5]:
def generate_prod_ind(
    indices: List[List[int]],
    select: Callable[[List[int]], bool] = lambda _: True,
    batch_size: int = None,
):
    no_elem = np.array(
        [len(elem) for elem in indices]
    )  # Highest order (z) + 1  - [x^0,x^1,...,x^z] -> z+1 terms
    tot_size = np.prod(
        no_elem
    )  # Total possible number of combinations (z_0*z_1*...*z_N)
    if batch_size is None:
        batch_size = tot_size
    # no_batches = (tot_size + batch_size - 1) // batch_size
    no_batches = int(np.ceil(tot_size / batch_size))
    for ibatch in range(no_batches):  # Loop over batches
        start_ind = ibatch * batch_size  # Start index from end index of previous batch.
        end_ind = np.minimum(start_ind + batch_size, tot_size)
        batch_ind = np.arange(start_ind, end_ind)  # Counting index
        multi_ind = np.array(
            np.unravel_index(batch_ind, no_elem)
        )  # N-dimensional indexes (N x 1D basis indexes)
        indices_out = np.array(
            [indices[icoo][multi_ind[icoo, :]] for icoo in range(len(indices))]
        ).T  # All possible N-dimensional indexes
        select_ind = np.where(
            np.asarray([select(elem) for elem in indices_out])
        )  # Truncation for combinations
        yield indices_out[select_ind], multi_ind[
            :, select_ind[0]
        ]  # Output indexes that adhere to the select truncation

Compute Taylor series expansion of the $G$-matrix and potential

$$
G_{\lambda,\mu}=\sum_{\mathbf{t}} g_{\mathbf{t},\lambda,\mu} \prod_i (r_i-r_i^\text{(eq)})^{t_i},
$$

$$
V=\sum_{\mathbf{t}} f_{\mathbf{t}} \prod_i (r_i-r_i^\text{(eq)})^{t_i},
$$

 where the derivative multi-index $\mathbf{t}$ is stored in `deriv_ind`, and the expansion coefficients $g_{\mathbf{t},\lambda,\mu}$ and $f_{\mathbf{t}}$ in `Gmat_coefs[:len(deriv_ind),:3N, :3N]` and `poten_coefs[:len(deriv_ind)]` respectively, where $N$ is the number of atoms


In [6]:
max_pow = 2  # Degree of Taylor Expansion
powers = [np.arange(max_pow + 1)] * ncoo  # 1D possible powers
tot_size = np.prod([len(p) for p in powers])  # Number of terms before truncation
batch_size = tot_size  # batch size - tot_size => one batch
no_batches = int(np.ceil(tot_size / batch_size))
select = lambda ind: np.sum(ind) <= max_pow  # Truncation criterion for Taylor expansion
gen_prod_ind = generate_prod_ind(powers, select=lambda ind: np.sum(ind) <= max_pow, batch_size=batch_size)
deriv_ind_list, deriv_mind_list = [], []  # Create empty lists to append to
for i in range(no_batches):  # Loop over batches
    _deriv_ind, _deriv_mind = next(gen_prod_ind)
    deriv_ind_list.append(_deriv_ind)  # Append indexes
    deriv_mind_list.append(_deriv_mind)

# Concatenate lists of indexes
deriv_ind, deriv_mind = np.concatenate(deriv_ind_list, axis=0), np.concatenate(
    deriv_mind_list, axis=1
)

print("Max expansion power:", max_pow)
print("Total number of terms before truncation:", tot_size)
print("Number of expansion terms after truncation:", len(deriv_ind))

Max expansion power: 2
Total number of terms before truncation: 729
Number of expansion terms after truncation: 28


In [7]:
print("Compute expansion of potential ...")
poten_coefs = deriv_list(potential, deriv_ind, r0, if_taylor=True)

Compute expansion of potential ...
Time for d= 0 : 32.51 s
Time for d= 1 : 26.48 s
Time for d= 2 : 63.88 s


In [8]:
print("Compute expansion of G-matrix ...")
Gmat_coefs = deriv_list(
    lambda x: Gmat(x, masses, internal_to_cartesian), deriv_ind, r0, if_taylor=True
)

Compute expansion of G-matrix ...
Time for d= 0 : 1.67 s
Time for d= 1 : 1.34 s
Time for d= 2 : 3.7 s


Define primitive Hermite basis functions $\mathcal{H}_n(x)$ and their derivatives $d\mathcal{H}_n(x)/dx$

In [9]:
def hermite(x, n):
    sqsqpi = np.sqrt(np.sqrt(np.pi))
    c = np.diag(1.0 / np.sqrt(2.0**n * factorial(n)) / sqsqpi) #Normalization constants
    f = hermval(np.asarray(x), c) * np.exp(-(x**2) / 2) #Hermite polynomial times square root of weight function
    return f.T


def hermite_deriv(x, n):
    sqsqpi = np.sqrt(np.sqrt(np.pi))
    c = np.diag(1.0 / np.sqrt(2.0**n * factorial(n)) / sqsqpi) #Normalization constants
    h = hermval(np.asarray(x), c) #Hermite polynomial 
    dh = hermval(np.asarray(x), hermder(c, m=1)) #Derivative of Hermite polynomial (H_l(x))
    f = (dh - h * x) * np.exp(-(x**2) / 2) #w'(x)H_l(x) + w(x)H_l'(x), where w: weight function, w(x) = np.exp(-x**2)
    return f.T

Define mapping between the coordinate $x\in (-\infty, \infty)$ of Hermite basis functions and the internal valence coordinates as $r_1=a_1 x_1 + b_1$, $r_2=a_2x_2+b_2$, $\alpha=a_3x_3+b_3$. The parameters $a_1,b_1,...,b_3$ are determined by mapping the vibrational Hamiltonian in valence coordinates onto the harmonic-oscillator Hamiltonian.

In [10]:
mask = deriv_ind != 0  # Mask for derivatives

# Vibrational G-matrix elements at equilibrium geometry
ind0 = np.where(mask.sum(axis=1) == 0)[0][0] #Equilibrium indexes
mu = np.diag(Gmat_coefs[ind0])[:ncoo]

# Second-order derivative of potential at equilibrium
ind2 = np.array(
    [   np.where((mask.sum(axis=1) == 1) & (deriv_ind[:, icoo] == 2))[0][0]
        for icoo in range(ncoo)]
)

freq = poten_coefs[ind2] * 2

# Linear mapping parameters
lin_a = jnp.sqrt(jnp.sqrt(mu / freq))
lin_b = r0

print("x->r linear mapping parameters 'a':", lin_a)
print("x->r linear mapping parameters 'b':", lin_b)

# Linear mapping function and its derivative
x_to_r_map = lambda x, icoo: lin_a[icoo] * x + lin_b[icoo]
jac_x_to_r_map = lambda x, icoo: np.ones_like(x) * lin_a[icoo]

x->r linear mapping parameters 'a': [0.05234934 0.11034365 0.11034365 0.15434545 0.15434545 0.30038996]
x->r linear mapping parameters 'b': [1.19999999 1.09999999 1.09999999 2.13       2.13       3.14159265]


Precompute matrix elements of expansion terms in Hermite basis, i.e., $\langle\mathcal{H}_i|r_\lambda^{t_\lambda}|\mathcal{H}_j\rangle$, $\langle\mathcal{H}_i|r_\lambda^{t_\lambda}|\partial_{r_\lambda}\mathcal{H}_j\rangle$, and $\langle\partial_{r_\lambda}\mathcal{H}_i|r_\lambda^{t_\lambda}|\partial_{r_\lambda}\mathcal{H}_j\rangle$, for three internal valence coordinates $\lambda =1..3$.

In [11]:
no_bas = [10] * ncoo #Number of primitive basis functions
npoints = [80] * ncoo #Number of quadrature points

prim_me = []
prim_dme = []
prim_d2me = []

for icoo in range(ncoo):
    x, w = hermgauss(npoints[icoo]) #Generate quadrature
    w /= np.exp(-(x**2))            #Divide by weight function, because it is a part of the basis
    r = x_to_r_map(x, icoo)         #Obtain r (coordinate values) from x (quadrature values)
    dr = r - r0[icoo]               #Displacement coordinate (subtraction of equilibrium geometry, r0)
    r_pow = dr[:, None] ** powers[icoo][None, :] #Generate powers of r
    psi = hermite(x, np.arange(no_bas[icoo] + 1))#Primitive basis functions
    dpsi = hermite_deriv(x, np.arange(no_bas[icoo] + 1)) / jac_x_to_r_map(x, icoo)[:, None] #Derivative of primitive basis functions
    me = jnp.einsum("gi,gt,gj,g->tij", psi, r_pow, psi, w)     #For Potential and p_i G_ij(q_k)p_j (averaging over q_k).
    dme = jnp.einsum("gi,gt,gj,g->tij", psi, r_pow, dpsi, w)   #For p_i G_ij p_j terms, j != i
    d2me = jnp.einsum("gi,gt,gj,g->tij", dpsi, r_pow, dpsi, w) #For p_i G_ii p_i terms (j=i)
    prim_me.append(me)
    prim_dme.append(dme)
    prim_d2me.append(d2me)

Solve contracted problems with basis |i>|0>|0> with i = 0...no_bas for each mode (averaging over lowest basis function for inactive modes).

In [54]:
contraction_sets = [[0],[1,2],[3,4],[5]] #Contraction sets
prim_poly_coefs = np.array([1, 1, 1, 1, 1, 1]) #Primitive polyadd numbers
pmax_sets = [no_bas[0],10,10,no_bas[5]]
_bas_ind = [np.arange(no_bas[icoo]) for icoo in range(ncoo)]

contr_vec = []
count = 0
for sets in contraction_sets:
    bas_ind = [np.arange(no_bas[icoo] + 1) if icoo_ in sets else np.arange(1) for icoo_ in range(ncoo)]
    bas_ind, bas_mind = next(generate_prod_ind(bas_ind,select=lambda ind: np.sum(ind * prim_poly_coefs) <= pmax_sets[count]))
    count = count + 1
    print('Set:',sets)
    print("total number of basis products:", len(bas_ind))

    me = np.prod(
        [
            prim_me[icoo_][
                np.ix_(deriv_mind[icoo_, :], bas_mind[icoo_, :], bas_mind[icoo_, :])
            ]
            for icoo_ in range(ncoo)
        ],
        axis=0,
    )

    vme = me.T @ poten_coefs

    gme = 0
    fac = {}
    for icoo in range(ncoo):
        for jcoo in range(ncoo):
            fac[icoo] = 1
            fac[jcoo] = -1
    
            if icoo != jcoo:
                dme = np.prod(
                    [
                        (
                            fac[icoo_]
                            * prim_dme[icoo_][
                                np.ix_(
                                    deriv_mind[icoo_, :],
                                    bas_mind[icoo_, :],
                                    bas_mind[icoo_, :],
                                )
                            ]
                            if icoo_ == icoo or icoo_ == jcoo
                            else prim_me[icoo_][
                                np.ix_(
                                    deriv_mind[icoo_, :],
                                    bas_mind[icoo_, :],
                                    bas_mind[icoo_, :],
                                )
                            ]
                        )
                        for icoo_ in range(ncoo)
                    ],
                    axis=0,
                )
            else:
                dme = np.prod(
                    [
                        (
                            prim_d2me[icoo_][
                                np.ix_(
                                    deriv_mind[icoo_, :],
                                    bas_mind[icoo_, :],
                                    bas_mind[icoo_, :],
                                )
                            ]
                            if icoo_ == icoo
                            else prim_me[icoo_][
                                np.ix_(
                                    deriv_mind[icoo_, :],
                                    bas_mind[icoo_, :],
                                    bas_mind[icoo_, :],
                                )
                            ]
                        )
                        for icoo_ in range(ncoo)
                    ],
                    axis=0,
                )
            gme += dme.T @ Gmat_coefs[:, icoo, jcoo]

    hme = 0.5 * gme + vme
    e, v = np.linalg.eigh(hme)
    print(e[0], e[0:10] - e[0])
    contr_vec.append(v)



Set: [0]
total number of basis products: 11
5989.644145445383 [    0.          1795.0533239   3590.1066478   5385.15997171
  7180.21329561  8975.26661951 10770.31994341 12565.37326731
 14360.42659252 16155.48205888]
Set: [1, 2]
total number of basis products: 66
5987.54975201514 [   0.         2964.11680031 3018.06403995 5928.23360062 5982.18084026
 6036.12807991 8892.35040093 8946.29764057 9000.24488022 9054.19211986]
Set: [3, 4]
total number of basis products: 66
5949.260044150832 [   0.         1309.18557136 1637.9787706  2617.52762174 2947.08990076
 3275.34704664 3925.064509   4255.37633901 4584.3969917  4912.11410064]
Set: [5]
total number of basis products: 11
5989.535360609758 [    0.          1215.37310202  2426.86228443  3634.44867331
  4838.11306799  6037.83919907  7233.6041187   8426.67676882
  9615.10924075 10809.62296162]


In [12]:
contr_vec = []
for icoo in range(ncoo):
    bas_ind = [
        np.arange(no_bas[icoo] + 1) if icoo_ == icoo else np.arange(1)
        for icoo_ in range(ncoo)
    ]
    bas_ind, bas_mind = next(generate_prod_ind(bas_ind))
    
    me = np.prod(
        [
            prim_me[icoo_][
                np.ix_(deriv_mind[icoo_, :], bas_mind[icoo_, :], bas_mind[icoo_, :])
            ]
            for icoo_ in range(ncoo)
        ],
        axis=0,
    )

    d2me = np.prod(
        [
            (
                prim_d2me[icoo_][
                    np.ix_(deriv_mind[icoo_, :], bas_mind[icoo_, :], bas_mind[icoo_, :])
                ]
                if icoo_ == icoo
                else prim_me[icoo_][
                    np.ix_(deriv_mind[icoo_, :], bas_mind[icoo_, :], bas_mind[icoo_, :])
                ]
            )
            for icoo_ in range(ncoo)
        ],
        axis=0,
    )

    vme = me.T @ poten_coefs
    gme = d2me.T @ Gmat_coefs[:, icoo, icoo]
    hme = 0.5 * gme + vme
    e, v = np.linalg.eigh(hme)
    print(icoo, e[0], e - e[0])
    contr_vec.append(v)

0 3433.1162189891284 [    0.          1794.40311262  3588.80622523  5383.20933785
  7177.61245046  8972.01556308 10766.41867569 12560.82178831
 14355.22490092 16149.62801354 17944.03112615]
1 3729.0932481839013 [    0.          2978.31122939  5956.62245879  8934.93368818
 11913.24491758 14891.55614697 17869.86737637 20848.17860576
 23826.48983516 26804.80106455 29783.11229394]
2 3729.093248201243 [    0.          2978.31122946  5956.62245893  8934.93368839
 11913.24491786 14891.55614732 17869.86737678 20848.17860625
 23826.48983571 26804.80106517 29783.11229464]
3 3363.07787336123 [    0.          1505.42938282  3010.15775822  4514.18431188
  6017.50823878  7520.12906957  9022.04770159 10523.3506244
 12027.20153236 13529.9575809  15045.33937193]
4 3363.077873363351 [    0.          1505.42938283  3010.15775823  4514.18431191
  6017.50823882  7520.12906961  9022.04770164 10523.35062446
 12027.20153243 13529.95758098 15045.33937202]
5 3292.6115092708274 [    0.          1215.44847204  24

Transform primitive matrix elements to the contracted basis

In [13]:
contr_me = [contr_vec[icoo].T @ prim_me[icoo] @ contr_vec[icoo] for icoo in range(ncoo)]
contr_dme = [contr_vec[icoo].T @ prim_dme[icoo] @ contr_vec[icoo] for icoo in range(ncoo)]
contr_d2me = [ contr_vec[icoo].T @ prim_d2me[icoo] @ contr_vec[icoo] for icoo in range(ncoo)]

Final solution 

In [15]:
pmax = 2
poly_coefs = np.array([2, 2, 2, 1, 1, 1])

bas_ind = [np.arange(np.ceil(pmax/poly_coefs[icoo]) + 1) for icoo in range(ncoo)]
bas_ind, bas_mind = next(
    generate_prod_ind(bas_ind, select=lambda ind: np.sum(ind * poly_coefs) <= pmax)
) #Polyad truncation: np.sum(ind * poly_coefs) <= pmax
print("polyad contraction number:", pmax)
print("total number of basis products:", len(bas_ind))

me = np.prod(
    [
        contr_me[icoo_][
            np.ix_(deriv_mind[icoo_, :], bas_mind[icoo_, :], bas_mind[icoo_, :])
        ]
        for icoo_ in range(ncoo)
    ],
    axis=0,
)

vme = me.T @ poten_coefs

gme = 0
fac = {}
for icoo in range(ncoo):
    for jcoo in range(ncoo):
        fac[icoo] = 1
        fac[jcoo] = -1

        if icoo != jcoo:
            dme = np.prod(
                [
                    (
                        fac[icoo_]
                        * contr_dme[icoo_][
                            np.ix_(
                                deriv_mind[icoo_, :],
                                bas_mind[icoo_, :],
                                bas_mind[icoo_, :],
                            )
                        ]
                        if icoo_ == icoo or icoo_ == jcoo
                        else contr_me[icoo_][
                            np.ix_(
                                deriv_mind[icoo_, :],
                                bas_mind[icoo_, :],
                                bas_mind[icoo_, :],
                            )
                        ]
                    )
                    for icoo_ in range(ncoo)
                ],
                axis=0,
            )
        else:
            dme = np.prod(
                [
                    (
                        contr_d2me[icoo_][
                            np.ix_(
                                deriv_mind[icoo_, :],
                                bas_mind[icoo_, :],
                                bas_mind[icoo_, :],
                            )
                        ]
                        if icoo_ == icoo
                        else contr_me[icoo_][
                            np.ix_(
                                deriv_mind[icoo_, :],
                                bas_mind[icoo_, :],
                                bas_mind[icoo_, :],
                            )
                        ]
                    )
                    for icoo_ in range(ncoo)
                ],
                axis=0,
            )
        gme += dme.T @ Gmat_coefs[:, icoo, jcoo]


hme = 0.5 * gme + vme
e, v = np.linalg.eigh(hme)
print(e[0], e - e[0])

polyad contraction number: 2
total number of basis products: 13
5948.743855619348 [   0.         1252.4796963  1381.18702282 1684.95745763 1851.7804148
 2467.5008949  2615.99346788 2751.76895052 2957.7549153  3003.97965958
 3026.95871591 3133.69027549 3415.2052134 ]
